In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kidney-stone-prediction-based-on-urine-analysis/kindey stone urine analysis.csv
/kaggle/input/playground-series-s3e12/sample_submission.csv
/kaggle/input/playground-series-s3e12/train.csv
/kaggle/input/playground-series-s3e12/test.csv


In [2]:
import sklearn
import matplotlib.pyplot as plt

In [3]:
train_full = pd.read_csv("/kaggle/input/playground-series-s3e12/train.csv")
test_full = pd.read_csv("/kaggle/input/playground-series-s3e12/test.csv")

In [4]:
train_full.head()

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1


In [5]:
train_full.describe()

,id,gravity,ph,osmo,cond,urea,calc,target
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,206.500000,1.017894,5.955459,651.545894,21.437923,278.657005,4.114638,0.444444
std,119.655756,0.006675,0.642260,234.676567,7.514750,136.442249,3.217641,0.497505
min,0.000000,1.005000,4.760000,187.000000,5.100000,10.000000,0.170000,0.000000
25%,103.250000,1.012000,5.530000,455.250000,15.500000,170.000000,1.450000,0.000000
50%,206.500000,1.018000,5.740000,679.500000,22.200000,277.000000,3.130000,0.000000
75%,309.750000,1.022000,6.280000,840.000000,27.600000,385.000000,6.630000,1.000000
max,413.000000,1.040000,7.940000,1236.000000,38.000000,620.000000,14.340000,1.000000


In [6]:
y_train = train_full.target
train_full.index = train_full["id"]
train_full.drop(["id","target"],axis =1, inplace = True)

test_full.index = test_full["id"]
test_X = test_full.drop(["id"],axis = 1)

In [7]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_full, y_train)

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score

my_model_1 = DecisionTreeClassifier(max_depth = 5,random_state=42)
my_model_1.fit(train_X,train_y)
val_probs_1 = my_model_1.predict_proba(val_X)
val_probs_1 = val_probs_1[:,1]
val_preds_1 = my_model_1.predict(val_X)

auc_score_1 = roc_auc_score(val_y, val_probs_1)
print(auc_score_1)

0.7018128005919348


In [9]:
from sklearn.ensemble import RandomForestClassifier

my_model_2 = RandomForestClassifier(random_state=42)
my_model_2.fit(train_X, train_y)
val_probs_2 = my_model_2.predict_proba(val_X)
val_probs_2 = val_probs_2[:,1]
val_preds_2 = my_model_2.predict(val_X)

auc_score_2 = roc_auc_score(val_y, val_probs_2)
print(auc_score_2)

0.7917129115797262


In [10]:
from xgboost import XGBClassifier

my_model_3 = XGBClassifier(n_estimators = 500, learning_rate = 0.05,random_state=42)
my_model_3.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(val_X, val_y)],verbose=False)
val_probs_3 = my_model_3.predict_proba(val_X)
val_probs_3 = val_probs_3[:,1]
val_preds_3 = my_model_3.predict(val_X)

auc_score_3 = roc_auc_score(val_y, val_probs_3)
print(auc_score_3)

0.7635960044395117


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


## Adding new training data to improve the performance of the models

In [11]:
add_data = pd.read_csv("/kaggle/input/kidney-stone-prediction-based-on-urine-analysis/kindey stone urine analysis.csv")

In [12]:
add_data.head()

,gravity,ph,osmo,cond,urea,calc,target
0,1.021,4.91,725,14.0,443,2.45,0
1,1.017,5.74,577,20.0,296,4.49,0
2,1.008,7.20,321,14.9,101,2.36,0
3,1.011,5.51,408,12.6,224,2.15,0
4,1.005,6.52,187,7.5,91,1.16,0


In [13]:
add_data.shape

(79, 7)

In [14]:
add_train_y = add_data.target
add_train_X = add_data.drop(["target"],axis = 1)

In [15]:
train_X_new = pd.concat([train_X, add_train_X], axis=0)
train_y_new = pd.concat([train_y, add_train_y], axis=0)

In [16]:
train_y_new.shape

(389,)

In [17]:
exp_model_1 = DecisionTreeClassifier(max_depth = 5,random_state=42)
exp_model_1.fit(train_X_new,train_y_new)
val_probs_1_new = exp_model_1.predict_proba(val_X)
val_probs_1_new = val_probs_1_new[:,1]
val_preds_1_new = exp_model_1.predict(val_X)

auc_score_1_new = roc_auc_score(val_y, val_probs_1_new)
print(auc_score_1_new)

0.7177210506844247


In [18]:
exp_model_2 = RandomForestClassifier(random_state=42)
exp_model_2.fit(train_X_new, train_y_new)
val_probs_2_new = exp_model_2.predict_proba(val_X)
val_probs_2_new = val_probs_2_new[:,1]
val_preds_2_new = exp_model_2.predict(val_X)

auc_score_2_new = roc_auc_score(val_y, val_probs_2_new)
print(auc_score_2_new)

0.8048464668886424


In [19]:
exp_model_3 = XGBClassifier(n_estimators = 500, learning_rate = 0.05,random_state=42)
exp_model_3.fit(train_X_new, train_y_new, early_stopping_rounds=5, eval_set=[(val_X, val_y)],verbose=False)
val_probs_3_new = exp_model_3.predict_proba(val_X)
val_probs_3_new = val_probs_3_new[:,1]
val_preds_3_new = exp_model_3.predict(val_X)

auc_score_3_new = roc_auc_score(val_y, val_probs_3_new)
print(auc_score_3_new)

0.7852386237513873


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


**We see that adding new training data improves the performance of all the three models, thus we will train our subsequent models using this dataset.**

## Hyper Parameter Tuning

Now,considering the three models auc_score, RandomForestClassifier has the best performance for the both the old and updated dataset, hence fine tuning the hyper parameters of the **RandomForestClassifier** and fitting it on the entire training data set.

In [20]:
from pprint import pprint

my_model = RandomForestClassifier()

print("Parameters currently in use: /n")
pprint(my_model.get_params())

Parameters currently in use: /n
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [21]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in the Random Forest
n_estimators = [int(x) for x in np.linspace(start=200,stop=2000,num=10)]

# Number of features to consider at every split
max_features = ["auto","sqrt"]

# Maximum number of levels in the tree
max_depth = [int(x) for x in np.linspace(start=10,stop=110,num=11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2,5,10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,4]

# Method of selecting samples for training each tree
bootstrap = ["True", "False"]

# Creating Random Grid
random_grid = {"n_estimators": n_estimators,
              "max_features": max_features,
              "max_depth": max_depth,
              "min_samples_split": min_samples_split,
              "min_samples_leaf": min_samples_leaf,
              "bootstrap": bootstrap}

In [22]:
from sklearn.model_selection import RandomizedSearchCV

#Randomly searching for parameters, using 3 fold cross validation, searching across 100 different 
#combinations

my_model_random = RandomizedSearchCV(estimator=my_model, param_distributions= random_grid, n_iter=100, cv=3,n_jobs = -1,random_state=42)
my_model_random.fit(train_X_new, train_y_new)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': ['True', 'False'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42)

In [23]:
my_model_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': 'True'}

In [24]:
val_probs_random = my_model_random.predict_proba(val_X)
val_probs_random = val_probs_random[:,1]
val_preds_random = my_model_random.predict(val_X)

auc_score_random = roc_auc_score(val_y, val_probs_random)
print(auc_score_random)

0.7976322604513504


In [25]:
from sklearn.model_selection import GridSearchCV

# Creating a parameter grid based on the results of the Random Grid Search

param_grid = {
    "n_estimators" : [100,400,1000,2000],
    "max_depth" : [20,80,100],
    "max_features" : [2,3,"sqrt"],
    "min_samples_split" : [2,8],
    "min_samples_leaf" : [1,3],
    "bootstrap" : ["False","True"]
}

# Creating a model
new_model = RandomForestClassifier()

# Instantiating the Grid Search Model
new_model_grid = GridSearchCV(estimator = new_model, param_grid = param_grid, cv=3,n_jobs=-1)

In [26]:
new_model_grid.fit(train_X_new,train_y_new)

new_model_grid.best_params_

{'bootstrap': 'True',
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [27]:
val_probs_grid = new_model_grid.predict_proba(val_X)
val_probs_grid = val_probs_grid[:,1]
val_preds_grid = new_model_grid.predict(val_X)

auc_score_grid = roc_auc_score(val_y, val_probs_grid)
print(auc_score_grid)

0.8035516093229744


In [28]:
# Creating a new parameter grid based on the first GridSearch results

param_grid = {
    "n_estimators" : [100,150,450,1200],
    "max_features" : ["sqrt"],
    "max_depth" : [20,70,80,90],
    "min_samples_split" : [2,7],
    "min_samples_leaf" : [1,2],
    "bootstrap" : ["True","False"]
}

model = RandomForestClassifier()

model_grid_2 = GridSearchCV(estimator = model,param_grid = param_grid, cv = 3, n_jobs = -1)

In [29]:
model_grid_2.fit(train_X_new, train_y_new)

model_grid_2.best_params_

{'bootstrap': 'True',
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 150}

In [30]:
val_probs_grid_2 = model_grid_2.predict_proba(val_X)
val_probs_grid_2 = val_probs_grid_2[:,1]
val_preds_grid_2 = model_grid_2.predict(val_X)

auc_score_grid_2 = roc_auc_score(val_y, val_probs_grid_2)
print(auc_score_grid_2)

0.8057713651498334


In [31]:
# Creating the final param_grid for Grid Search based on the previous results

param_grid = {
    "n_estimators" : [100,150,200],
    "max_features" : ["sqrt"],
    "max_depth" : [70],
    "min_samples_split" : [2,5,8],
    "min_samples_leaf" : [2,3],
    "bootstrap" : ["False","True"]
}

model_grid = RandomForestClassifier()

model_grid_3 = GridSearchCV(estimator = model_grid, param_grid = param_grid, cv = 3, n_jobs = -1)

In [32]:
model_grid_3.fit(train_X_new, train_y_new)
model_grid_3.best_params_

{'bootstrap': 'True',
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [33]:
val_probs_grid_3 = model_grid_3.predict_proba(val_X)
val_probs_grid_3 = val_probs_grid_3[:,1]
val_preds_grid_3 = model_grid_3.predict(val_X)

auc_score_grid_3 = roc_auc_score(val_y, val_probs_grid_3)
print(auc_score_grid_3)

0.802441731409545


## Fitted Hyperparameters 

Taking the final grid search, the hyper parameters fine-tuned for this model are: 
* n_estimators = 100
* max_depth = 70
* max_features = "sqrt"
* min_samples_split = 2
* min_samples_leaf = 2
* bootstrap = False

Now, training a model with these hyperparameters on the entire training data adding the external data source

In [34]:
# Creating the final training dataset

final_train_X = pd.concat([train_full,add_train_X],axis = 0)
final_train_y = pd.concat([y_train,add_train_y],axis = 0)

In [35]:
final_model = RandomForestClassifier(n_estimators=100, max_depth=70, max_features="sqrt",min_samples_split=3, min_samples_leaf=2, bootstrap=False)

final_model.fit(final_train_X, final_train_y)

RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=3)

In [36]:
final_proba = final_model.predict_proba(test_X)

In [37]:
final_proba.shape

(276, 2)

In [38]:
output = pd.DataFrame()
output["id"] = range(414,690)
output["prob"] = final_proba[:,1]

In [39]:
output

,id,prob
0,414,0.311667
1,415,0.641667
2,416,0.958333
3,417,0.400833
4,418,0.392500
...,...,...
271,685,0.805000
272,686,0.025000
273,687,0.713000
274,688,0.006667


In [40]:
output.to_csv("final-submission.csv", index=False)